## 有监督微调

上一章介绍了分布式训练基本概念、分布式策略，以及Deepspeed是如何加速模型的训练。本章将介绍有监督微调（Supervised Fine-Tuning, SFT），也就是在已经训练好的模型基础上，使用有标注的特定任务数据进行下一步的微调，从而使模型具备遵循指令的能力。  


### 提示学习和语境学习

#### 提示学习

提示学习不同于传统的监督学习，直接利用在大量原始文本上进行预训练的语言模型，通过定义提示函数，使模型可以执行小样本甚至零样本学习。  

提示学习流程非常简洁，可以描述为三个阶段：提示添加、答案搜索和答案映射:  

<img src="./images/提示学习.png" style="zoom:60%;" /> 

具体来说，原始输入`x`经过一个模板，被构造成一个提示，然后将其输入语言模型，语言模型即以概率的方式填充模版中待填充的内容，然后根据模型的输出即可得到最终的预测标签。

#### 语境学习

语境学习也称上下文学习，是指模型可以从上下文的几个例子中学习：向模型输入特定任务的一些具体例子以及要测试的样例，模型可以根据给定的实力续写出测试样例的答案。语境学习可以看作是提示学习的一个子类。语境学习的关键思想是从类比中学习，整个过程并不需要对模型进行参数更新，仅执行向前的推理。  

<img src="./images/语境学习.png" style="zoom:60%;" /> 

### 高效模型微调

由于大模型的参数量十分庞大，当其应用到下游任务时，微调全部参数需要相当高的算力，因此为了节省成本，研究人员提出了多种参高效的微调方法，旨在仅训练少量参数使模型适应到下游任务。接下来主要是介绍LoRA，LoRA方法可以在缩减训练参数量和GPU显存占用的同时，使训练后的模型具有与全量微调相当的性能。  

#### LoRA

研究人员认为参数更新量即使投影到较小的子空间中，也不会影响到学习的有效性。因此，提出固定预训练模型参数不变，在原本权重矩阵旁路添加低秩矩阵的乘积作为可训练参数，用以模拟参数的变化量，LoRA方法的计算流程图为:  

<img src="./images/LoRA结构图.png" style="zoom:60%;" /> 

具体来说，假设预训练权重为$W_0$(d * k)，可以通过可训练参数$\Delta W = BA $变为(d * r + r * d)，大大的减少了训练参数的子空间，公式为:  

$$
h = W_0 x + \Delta W x = W_0 x + B A x
$$

对于使用LoRA的模型来说，由于可以将原始权重与训练后权重合并，即$W = W_0 + B A$，因此在推理时不存在额外的开销。

peft库中含有LoRA在内的多种高效微调方法，且与transformer兼容示例为: 

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

'''
在Lora的配置项中有两个参数: r和lora_alpha
r: Lora attention dimension, r即rank 是矩阵的秩(rank), 用于控制LoRA中低秩矩阵的维度
lora_alpha: 是一个放缩系数，用于控制微调过程中更新矩阵 ΔW 的大小, 也就是控制其对原始模型的影响
'''

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)